In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 
import time
import random 
import numpy as np
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

from keras.datasets import mnist
#import matplotlib
#%matplotlib inline
#import matplotlib.pyplot as plt
#import l2_attack
import keras
#from defense import *
import tensorflow as tf
from wrapper import Model


In [6]:
class blackbox:
    def __init__(self,model):
        self.model = model
        
    def attack_untargeted(self, x0, y0, alpha = 0.2, beta = 0.001, iterations = 1000):
        """ Attack the original image and return adversarial example
            model: (pytorch model)
            train_dataset: set of training data
            (x0, y0): original image
        """

        if (self.model.predict(x0) != y0):
            print("Fail to classify the image. No need to attack.")
            return x0,0

        num_directions = 1000
        best_theta, g_theta = None, float('inf')
        query_count = 0

        timestart = time.time()
        for i in range(num_directions):
            theta = torch.randn(x0.shape).type(torch.FloatTensor)
            initial_lbd = torch.norm(theta)
            theta = theta/torch.norm(theta)
            if self.model.predict(x0+np.array(initial_lbd*theta)) != y0:
                query_count += 1
                lbd, count = self.fine_grained_binary_search( x0, y0, theta, initial_lbd, g_theta)
                query_count += count
                if lbd < g_theta:
                    best_theta, g_theta = theta,lbd
                    print("--------> Found distortion %.4f" % g_theta)

        timeend = time.time()
        print("==========> Found best distortion %.4f in %.4f seconds using %d queries" % (g_theta, timeend-timestart, query_count))



        timestart = time.time()
        g1 = 1.0
        theta, g2 = best_theta.clone(), g_theta
        torch.manual_seed(0)
        opt_count = 0
        stopping = 0.01
        prev_obj = 100000
        for i in range(iterations):
            if g_theta < 1:
                print("====================query number after distortion < 1 =======================: ",opt_count)
                break
                
            gradient = torch.zeros(theta.size())
            q = 10
            min_g1 = float('inf')
            for _ in range(q):
                u = torch.randn(theta.size()).type(torch.FloatTensor)
                u = u/torch.norm(u)
                ttt = theta+beta * u
                ttt = ttt/torch.norm(ttt)
                g1, count = self.fine_grained_binary_search_local( x0, y0, ttt, initial_lbd = g2, tol=beta/500)
                opt_count += count
                gradient += (g1-g2)/beta * u
                if g1 < min_g1:
                    min_g1 = g1
                    min_ttt = ttt
            gradient = 1.0/q * gradient

            if (i+1)%50 == 0:
                print("Iteration %3d: g(theta + beta*u) = %.4f g(theta) = %.4f distortion %.4f num_queries %d" % (i+1, g1, g2, torch.norm(g2*theta), opt_count))
                if g2 > prev_obj-stopping:
                    break
                prev_obj = g2

            min_theta = theta
            min_g2 = g2

            for _ in range(15):
                new_theta = theta - alpha * gradient
                new_theta = new_theta/torch.norm(new_theta)
                new_g2, count = self.fine_grained_binary_search_local(x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                opt_count += count
                alpha = alpha * 2
                if new_g2 < min_g2:
                    min_theta = new_theta 
                    min_g2 = new_g2
                else:
                    break

            if min_g2 >= g2:
                for _ in range(15):
                    alpha = alpha * 0.8
                    new_theta = theta - alpha * gradient
                    new_theta = new_theta/torch.norm(new_theta)
                    new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                    opt_count += count
                    if new_g2 < g2:
                        min_theta = new_theta 
                        min_g2 = new_g2
                        break

            if min_g2 <= min_g1:
                theta, g2 = min_theta, min_g2
            else:
                theta, g2 = min_ttt, min_g1

            if g2 < g_theta:
                best_theta, g_theta = theta.clone(), g2

            #print(alpha)
            if alpha < 1e-4:
                alpha = 1.0
                print("Warning: not moving, g2 %lf gtheta %lf" % (g2, g_theta))
                beta = beta * 0.1
                if (beta < 0.0005):
                    break

        target = model.predict(x0 + g_theta*best_theta)
        timeend = time.time()
        print("\nAdversarial Example Found Successfully: distortion %.4f target %d queries %d \nTime: %.4f seconds" % (g_theta, target, query_count + opt_count, timeend-timestart))
        return  g_theta*best_theta,query_count+opt_count 
    def fine_grained_binary_search_local(self, x0, y0, theta, initial_lbd = 1.0, tol=1e-5):
        nquery = 0
        lbd = initial_lbd

        if self.model.predict(x0+lbd*theta) == y0:
            lbd_lo = lbd
            lbd_hi = lbd*1.01
            nquery += 1
            while self.model.predict(x0+lbd_hi*theta) == y0:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 20:
                    return float('inf'), nquery
        else:
            lbd_hi = lbd
            lbd_lo = lbd*0.99
            nquery += 1
            while self.model.predict(x0+lbd_lo*theta) != y0 :
                lbd_lo = lbd_lo*0.99
                nquery += 1

        while (lbd_hi - lbd_lo) > tol:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + lbd_mid*theta) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery

    def fine_grained_binary_search(self, x0, y0, theta, initial_lbd, current_best):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict(x0+current_best*theta) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd

        lbd_hi = lbd
        lbd_lo = 0.0

        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + lbd_mid*theta) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery



In [ ]:
session = keras.backend.get_session()
keras.backend.set_learning_phase(False)
model = keras.models.load_model("data/mnist")
model = Model(model,[0.0,1.0])

attack = blackbox(model)

#touse = [x for x in tf.trainable_variables() if 'Generator' in x.name]
#saver = tf.train.Saver(touse)
#saver.restore(session, 'data/mnist-gan')

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = np.array(x_test, dtype=np.float32)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test /= 255.0
#image = x_test[:1]
#adv = attack.attack_untargeted(image[0],y_test[0], alpha = 4, beta = 0.005, iterations = 1000)
count = []
for i in range(20):
    label = model.predict(x_test[i])
    if label == y_test[i]:
        count.append(1)
    else:
        count.append(0)
    
print("accuracy of this model is:", sum(count)/len(count))

accuracy of this model is: 0.95


In [ ]:
dist = []
count = []
for i in range(15):
    print("=========================image ",i+1,"==========================================")
    print("true label:",y_test[i+20])
    print("predicted label:",model.predict(x_test[i+20]))
    adv_mod, queries= attack.attack_untargeted(x_test[i+20],y_test[i+20], alpha = 4, beta = 0.005, iterations = 1000)
    dist.append(np.linalg.norm(adv_mod))
    count.append(queries)

index = np.nonzero(count)
index = list(index)[0].tolist()

avg_distortion = np.mean(np.array(dist)[index])
avg_count = np.mean(np.array(count)[index])
print("the average distortion for %2d images :"%(len(index)),avg_distortion)
for i in dist:
    print(i)
print("the number of queries for %2d images :"%(len(index)), avg_count)
for j in count:
    print(j)

=========================image  1 ==========================================
true label: 9
predicted label: 9
--------> Found distortion 14.1668
--------> Found distortion 10.9718
--------> Found distortion 10.4159
--------> Found distortion 8.9514
--------> Found distortion 7.8105
--------> Found distortion 7.2462
==========> Found best distortion 7.2462 in 4.2515 seconds using 2107 queries
Iteration  50: g(theta + beta*u) = 1.9751 g(theta) = 1.9774 distortion 1.9774 num_queries 17310
Iteration 100: g(theta + beta*u) = 1.3903 g(theta) = 1.3903 distortion 1.3903 num_queries 32012
Iteration 150: g(theta + beta*u) = 1.1774 g(theta) = 1.1774 distortion 1.1774 num_queries 44898
Iteration 200: g(theta + beta*u) = 1.0814 g(theta) = 1.0814 distortion 1.0814 num_queries 56249
Iteration 250: g(theta + beta*u) = 1.0156 g(theta) = 1.0155 distortion 1.0155 num_queries 67238
====================query number after distortion < 1 =======================:  71982

Adversarial Example Found Successfully

--------> Found distortion 15.4228
--------> Found distortion 15.3293
--------> Found distortion 13.6165
--------> Found distortion 12.7040
--------> Found distortion 11.1775
--------> Found distortion 10.8304
==========> Found best distortion 10.8304 in 4.0858 seconds using 2100 queries
Iteration  50: g(theta + beta*u) = 3.9726 g(theta) = 3.9712 distortion 3.9712 num_queries 16285
Iteration 100: g(theta + beta*u) = 3.0883 g(theta) = 3.0882 distortion 3.0882 num_queries 30802
Iteration 150: g(theta + beta*u) = 2.6740 g(theta) = 2.6743 distortion 2.6743 num_queries 44176
Iteration 200: g(theta + beta*u) = 2.4481 g(theta) = 2.4474 distortion 2.4474 num_queries 56778
Iteration 250: g(theta + beta*u) = 2.3067 g(theta) = 2.3064 distortion 2.3064 num_queries 68453
Iteration 300: g(theta + beta*u) = 2.1957 g(theta) = 2.1955 distortion 2.1955 num_queries 79981
Iteration 350: g(theta + beta*u) = 2.1239 g(theta) = 2.1242 distortion 2.1242 num_queries 91192
Iteration 400: g(theta + beta*u) = 2.07

Iteration 250: g(theta + beta*u) = 1.6239 g(theta) = 1.6239 distortion 1.6239 num_queries 66505
Iteration 300: g(theta + beta*u) = 1.5518 g(theta) = 1.5518 distortion 1.5518 num_queries 77120
Iteration 350: g(theta + beta*u) = 1.5077 g(theta) = 1.5073 distortion 1.5073 num_queries 87711
Iteration 400: g(theta + beta*u) = 1.4712 g(theta) = 1.4710 distortion 1.4710 num_queries 97688
Iteration 450: g(theta + beta*u) = 1.4485 g(theta) = 1.4485 distortion 1.4485 num_queries 107563
Iteration 500: g(theta + beta*u) = 1.4259 g(theta) = 1.4258 distortion 1.4258 num_queries 117388
Iteration 550: g(theta + beta*u) = 1.4103 g(theta) = 1.4103 distortion 1.4103 num_queries 127067
Iteration 600: g(theta + beta*u) = 1.3998 g(theta) = 1.3998 distortion 1.3998 num_queries 136634
Iteration 650: g(theta + beta*u) = 1.3920 g(theta) = 1.3920 distortion 1.3920 num_queries 146118

Adversarial Example Found Successfully: distortion 1.3920 target 8 queries 148311 
Time: 286.5119 seconds
========================